In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import anndata
import numpy as np
import pickle
from moscot.framework.estimators import LineageEstimator

In [4]:
adata_dir = "/Users/DominikKlein/Documents/PhD/data/Hu_zebrafish_linnaeus/adata.h5ad"
trees_dir = "/Users/DominikKlein/Documents/PhD/data/Hu_zebrafish_linnaeus/trees.pkl"

In [5]:
adata = anndata.read(adata_dir)

In [6]:
# focus on time-points ctrl 3, 7 (H5-> ctrl, Hr26 -> 3, Hr2->7)
dpis_orig_ident = ['H5', 'Hr26', 'Hr2a','Hr2b']
dpis = [0, 3, 7, 7]
dpis_unq = np.unique(dpis)
dpis_str =['ctrl', 'dpi3', 'dpi7']
dpis_trees = ['H5', 'Hr26', 'Hr2']
core_nodes = [100, 100, 100]

In [7]:
adata_filtered = adata[adata.obs['orig.ident'].isin(dpis_orig_ident)].copy()
adata_filtered

AnnData object with n_obs × n_vars = 49249 × 31466
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mito', 'time', 'morphine', 'AV', 'inhib', 'so.far.ident.subtypes', 'so.far.ident', 'RNA_snn_res.4', 'seurat_clusters', 'work.ident', 'fibro.subtypes', 'is.inhib', 'split.inhib', 'zoom.ident', 'zoom.subtypes', 'plot.ident', 'CM.ident', 'paper.ident', 'RNA_snn_res.2', 'RNA_snn_res.3', 'RNA_snn_res.3.5', 'work.ident2', 'work.ident3', 'final.zoom', 'lineage.ident', 'Cell_type'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'trees'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [8]:
adata_filtered.obs["dpi"] = adata_filtered.obs["orig.ident"].replace(dpis_orig_ident, dpis).astype("category")

In [9]:
with open(trees_dir, 'rb') as file:
    trees = pickle.load(file)

In [10]:
trees_dpis = {}
trees_dpis_core = {}
for i, dpi in enumerate(dpis_unq):
    trees_dpis[dpi] = trees[dpis_trees[i]]
    trees_dpis_core[dpi] = trees_dpis[dpi].subgraph([str(n) for n in range(core_nodes[i])])


In [11]:
key = "depth"
epsilon = 0.2

In [27]:
estimator = LineageEstimator(adata_filtered, "dpi", trees_dpis_core, epsilon)

In [28]:
estimator.prepare((0,3))

In [ ]:
ot_results = estimator.fit()

In [ ]:
self._solver_dict[tup].fit(self.geometries_inter_dict[tup], self.geometries_intra_dict[tup[0]], 
                           self.geometries_intra_dict[tup[1]], self.a_dict[tup], self.b_dict[tup])